In [3]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from scrapy.selector import Selector
from sqlalchemy.sql import text as sa_text

engine = create_engine('mysql+pymysql://root:@localhost/oerintegrationdb', echo=False)
Base = declarative_base(engine)


class Triple(Base):
    __tablename__ = 'triple'
    __table_args__ = {'autoload': True}

# 
class CleanTriple(Base):
    __tablename__ = 'cleantriple'
    __table_args__ = {'autoload': True}

metadata = Base.metadata
Session = sessionmaker(bind=engine)
session = Session()

print("Ready DB")

Ready DB


In [4]:
triples = session.query(Triple).filter(Triple.subject =="feedbooks")
#engine.execute(sa_text('''TRUNCATE TABLE cleantriple''').execution_options(autocommit=True))

print("Cleaned")


Cleaned


In [5]:
source_raw = "feedbooks"
session.add(CleanTriple(subject_uri="http://es.feedbooks.com" ,subject_id =source_raw, predicate="hasName", object="Feedbooks"))
session.add(CleanTriple(subject_uri="http://es.feedbooks.com" ,subject_id =source_raw, predicate="isA", object="Site"))
session.add(CleanTriple(subject_uri="http://es.feedbooks.com" ,subject_id =source_raw, predicate="hasSiteLink", object="http://es.feedbooks.com"))

for triple in triples:
    # print(triple.subject_uri=source ,subject_id )
    body = triple.object
    source = triple.source
    # Setting name
    name = Selector(text=body).xpath('//h1/text()').get()
    name_triple =  name.replace(" ", "")
    name_triple =  name_triple.replace("/", "")
    name_triple =  name_triple.replace(":", "")
    # print(name_triple)
    session.add(CleanTriple(subject_uri=source ,subject_id =source_raw, predicate="hasBook", object=name_triple))
    session.add(CleanTriple(subject_uri=source ,subject_id =name_triple, predicate="hasTitle", object=name))
    # print(triple.source)
    session.add(CleanTriple(subject_uri=source ,subject_id =name_triple, predicate="hasSourceLink", object=triple.source))
    session.add(CleanTriple(subject_uri=source ,subject_id =name_triple, predicate="materialType", object="Book"))
    
    author = Selector(text=body).xpath('//h2/em/a/text()').get()
    # print(author)
    session.add(CleanTriple(subject_uri=source ,subject_id =name_triple, predicate="hasAuthor", object=author))
    
    link = Selector(text=body).xpath('//h3/a/@href').getall()
    # print(link) 
    session.add(CleanTriple(subject_uri=source ,subject_id =name_triple, predicate="hasDownloadLink", object=link[0])) 
    
    about_aux = Selector(text=body).xpath("//div[@class='indent justify']/p/text()").getall()
    about = ""
    for aux in about_aux:
        about += aux
    # print(about) 
    session.add(CleanTriple(subject_uri=source ,subject_id =name_triple, predicate="hasAbout", object=about))
    
    
    meta = Selector(text=body).xpath("//strong/..").getall()
    meta = meta[:len(meta)-5]
    
    for meta_iter in meta:
        meta_class = Selector(text=meta_iter).xpath("//text()").get()
        # print(meta_class)
        if meta_class == "Language:":
            language = Selector(text=meta_iter).xpath("//a/text()").get()
            # print(language)
            session.add(CleanTriple(subject_uri=source ,subject_id =name_triple, predicate="hasLanguage", object=language))
        elif meta_class == "Published in:":
            publication_date = Selector(text=meta_iter).xpath("//a/text()").get()
            # print(publication_date)
            session.add(CleanTriple(subject_uri=source ,subject_id =name_triple, predicate="hasPublicationDate", object=publication_date))
        elif meta_class == "Word count:":
            word_count_aux = Selector(text=meta_iter).xpath("//p/text()").get()
            # print(word_count_aux)
            word_count_aux =  word_count_aux.split("(")
            # print(word_count_aux)
            word_count = word_count_aux[0].strip()
            word_count = word_count[0:word_count.find(" ")].replace(",","")
            # print(word_count)
            session.add(CleanTriple(subject_uri=source ,subject_id =name_triple, predicate="hasNumberOfWords", object=word_count))
            time = int(list(filter(str.isdigit, word_count_aux[1]))[0])
            # print(time)
            session.add(CleanTriple(subject_uri=source ,subject_id =name_triple, predicate="hasNumberOfHours", object=time))
        elif meta_class == "Source:":
             # print(meta_iter)
             book_source = Selector(text=meta_iter).xpath("//a/@href").get()
             # print(book_source)
             if book_source is not None:
                session.add(CleanTriple(subject_uri=source ,subject_id =name_triple, predicate="hasBookSource", object=book_source))
        elif meta_class == "Translator:":
             # print(meta_iter)
             translator = Selector(text=meta_iter).xpath("//p/text()").get()
             # print(translator.strip())
             session.add(CleanTriple(subject_uri=source ,subject_id =name_triple, predicate="hasTranslator", object=translator.strip()))
        else:
             # print(triple.source)
             # print(meta_iter)
            meta_class = Selector(text=meta_iter).xpath("//p/strong/text()").get()
            # print(meta_class)
            if meta_class == "License:":
                 # print(meta_iter)
                 license = Selector(text=meta_iter).xpath("//p/a/@href").get()
                 # print(license)
                 session.add(CleanTriple(subject_uri=source ,subject_id =name_triple, predicate="hasLicense", object=license))
            # elif meta_class == "Copyright:":
            #      print(meta_iter)
            #      copyright = Selector(text=meta_iter).xpath("//p/text()").get()
            #      print(copyright)
                 # link_copyright = Selector(text=meta_iter).xpath("//a").get()
                 # if link_copyright:
                 #     print(link_copyright)
                 # session.add(CleanTriple(subject_uri=source ,subject_id =name_triple, predicate="hasCopyright", object=copyright))
            
    categories = Selector(text=body).xpath("//div[@class='span-15 prepend-top book_categories']/span/a/text()").getall()
    # print(categories)
    if len(categories) > 0:
        for cat in categories:
            # print(cat)
            session.add(CleanTriple(subject_uri=source ,subject_id =name_triple, predicate="hasCategory", object=cat))
    
    image = Selector(text=body).xpath("//img[@class='cover book_page_cover']/@src").get()
    # print(image)
    session.add(CleanTriple(subject_uri=source ,subject_id =name_triple, predicate="hasImageLink", object=image))



session.commit()

print("Done!")

Done!
